# Similarity between the city

# import zone

In [1]:
import gensim
import os
import collections
import smart_open
import random
import pickle
import pandas as pd
import copy
import re

C:\Users\sociocom\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## parameter zone

In [2]:
path_play = r'C:\Users\sociocom\Desktop\Crawler\backup\playing'

In [3]:
boundary = 10

In [4]:
grid_size = 15
diff = 0.1/grid_size

In [5]:
bkk_lat = 13.685780
bkk_long = 100.484605
diff_wid = 0.1/grid_size

In [6]:
kyt_lat = 34.955205 #35.005205
kyt_long = 135.675300 #135.7353
diff_wid = (135.795300 - 135.675300)/grid_size
#135.675300,34.955205,135.795300,35.055205

In [7]:
color = ['#56000a','orange','gray','black','brown','pink', 'purple','yellow','red','green','blue','#693487','#001994','#7668bb','#d0dc62','#936c6c','#005360',
        '#0d92e8','#98fefb','#e552e5','#fd130c','#660585','#754cda','#cadb11','#62ab12','#cadb11','#772838','#c0b018','#69002c','#76539f',
         '#b8bca8','#fd5620','#635782','#3e520a','#2c63c2','#7be491','#7a1154','#b45a30','#9270d8','#dbb8df']

In [8]:
len(color)

40

In [74]:
color = ['#56000a','orange','gray','black','brown','pink', 'purple','yellow','red','green','blue','#001994','#69002c','#936c6c',
         '#76539f','#005360','#0d92e8','#98fefb','#e552e5','#fd130c','#660585','#754cda','#cadb11','#7668bb','#62ab12','#772838',
'#c0b018','#b8bca8','#fd5620','#635782','#3e520a','#2c63c2','#7be491','#7a1154','#b45a30','#9270d8','#d0dc62','#693487','#dbb8df']

## function zone

In [11]:
def save_obj(file, obj):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [12]:
def clean_place(df, name):
    return df[df['place']!=name]

In [13]:
def separate_loc(df, flat, flong):
    df['ex_lat'] = df['ex_lat'].apply(lambda x: float(x))
    df['ex_long'] = df['ex_long'].apply(lambda x: float(x))
    data = []
    lat = flat
    long = flong
    for i in range(grid_size):
        for j in range(grid_size):
            data.append(df[(df['ex_lat'] >= lat) & (df['ex_lat'] < lat+diff) & (df['ex_long'] >= long) & (df['ex_long'] < long + diff_wid)])
            idx = []
            idx.append((long, lat))
            idx.append((long+diff_wid, lat))
            #print(lat," ",long)
            lat += diff
            idx.append((long, lat))
            idx.append((long+diff_wid, lat))
            u = i*grid_size + j
            idx.append(str(u))
            idx_color[str(u)] = idx
        long += diff_wid
        lat = flat
    return data

In [14]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="utf-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [15]:
def in_tuple(tp, x):
    return [z for y,z in tp if y == x]

In [16]:
def merge_area(threshold, idx_color,city_list):
    for i in range(len(idx_color)):
        w = i-1 if (i-1)%grid_size != 9 else -1
        e = i+1 if ((i+1)%grid_size != 0) and (i+1 < len(idx_color)) else -1
        n = i-grid_size
        s = i+grid_size if i+grid_size < len(idx_color) else -1
        #if len(city_list[i])<boundary:
        #    idx_color[str(i)][4] = "-1"
        #else:
        """if w>0: #and len(city_list[w])>=boundary:
            if (len(city_list[i])+len(city_list[w])) > 0:
                similarw = (in_tuple(all_sims[i], w)[0] + in_tuple(all_sims[w], i)[0])/(2*(len(city_list[i])+len(city_list[w])))
                if similarw > threshold:
                    idx_color[str(w)][4] = idx_color[str(i)][4]
            else:
                idx_color[str(w)][4] = idx_color[str(i)][4]
        if n>0: #and len(city_list[n])>=boundary:
            if (len(city_list[i])+len(city_list[n])):
                similarn = (in_tuple(all_sims[i], n)[0] + in_tuple(all_sims[n], i)[0])/(2*(len(city_list[i])+len(city_list[n])))
                if similarn > threshold:
                    idx_color[str(n)][4] = idx_color[str(i)][4]
            else:
                idx_color[str(n)][4] = idx_color[str(i)][4]
        """
        if e>0:# and len(city_list[e])>=boundary:
            if (len(city_list[i])+len(city_list[e])) > 0:
                similare = (in_tuple(all_sims[i], e)[0] + in_tuple(all_sims[e], i)[0])/(2*(len(city_list[i])+len(city_list[e])))
                if similare > threshold:
                    idx_color[str(e)][4] = idx_color[str(i)][4]
            else:
                idx_color[str(e)][4] = idx_color[str(i)][4]
        if s>0:# and len(city_list[s])>=boundary:
            if (len(city_list[i])+len(city_list[s])) > 0:
                similars = (in_tuple(all_sims[i], s)[0] + in_tuple(all_sims[s], i)[0])/(2*(len(city_list[i])+len(city_list[s])))
                if similars > threshold:
                    idx_color[str(s)][4] = idx_color[str(i)][4]
            else:
                idx_color[str(s)][4] = idx_color[str(i)][4]
    return idx_color

In [17]:
def merge_area(threshold, idx_color,city_list):
    for i in range(len(idx_color)):
        w = i-1 if (i-1)%10 != (grid_size-1) else -1
        e = i+1 if ((i+1)%10 != 0) and (i+1 < len(idx_color)) else -1
        n = i-10 
        s = i+10 if i+10 < len(idx_color) else -1
        if len(city_list[i])<boundary:
            idx_color[str(i)][4] = "-1"
        else:
            if w>0 and len(city_list[w])>=boundary:
                similarw = (in_tuple(all_sims[i], w)[0] + in_tuple(all_sims[w], i)[0])/(2*(len(city_list[i])+len(city_list[w])))
                if similarw > threshold:
                    idx_color[str(w)][4] = idx_color[str(i)][4]
            if n>0 and len(city_list[n])>=boundary:
                similarn = (in_tuple(all_sims[i], n)[0] + in_tuple(all_sims[n], i)[0])/(2*(len(city_list[i])+len(city_list[n])))
                if similarn > threshold:
                    idx_color[str(n)][4] = idx_color[str(i)][4]
            if e>0 and len(city_list[e])>=boundary:
                similare = (in_tuple(all_sims[i], e)[0] + in_tuple(all_sims[e], i)[0])/(2*(len(city_list[i])+len(city_list[e])))
                if similare > threshold:
                    idx_color[str(e)][4] = idx_color[str(i)][4]
            if s>0 and len(city_list[s])>=boundary:
                similars = (in_tuple(all_sims[i], s)[0] + in_tuple(all_sims[s], i)[0])/(2*(len(city_list[i])+len(city_list[s])))
                if similars > threshold:
                    idx_color[str(s)][4] = idx_color[str(i)][4]
    return idx_color

In [18]:
def count_areas(idx_color):
    areas = set()
    for i in range(grid_size):
        for j in range(grid_size):
            areas.add(idx_color[str((i*grid_size)+j)][4])
    return len(areas)

In [19]:
def print_areas(idx_color):
    areas = set()
    for i in range(grid_size):
        s = ""
        for j in range(grid_size):
            idx = str((i*grid_size)+j)
            if int(idx_color[idx][4]) < grid_size and int(idx_color[idx][4]) >= 0:
                s += " "
            s += str(idx_color[idx][4]) +" "
            areas.add(idx_color[idx][4])
        print(s)
    print("\n #areas :" , len(areas))
    return areas

In [20]:
def print_volumn(list_place):
    for i in range(grid_size):
        s = ""
        for j in range(grid_size):
            idx = (i*10)+j
            if len(list_place[idx]) < 99 and len(list_place[idx]) >= 0:
                s += " "
                if len(list_place[idx]) <10:
                    s+= " "
            s += str(len(list_place[idx])) +" "
        print(s)

## data zone 

### Kyoto 

In [24]:
kyt_all = load_obj(path_play+r"\kyt_data2")

In [25]:
kyt_idx, kyt, k_rank, k_sims = kyt_all

In [26]:
kyt[9]

,acc,acc_id,created_time,ex_lat,ex_long,lang,lot_ne,lot_sw,place,text,tweet_id,tweet_reply_id,tweet_reply_user_id,tweet_reply_username
144,shiyugin_rec,2795245832,Fri Jun 01 02:56:25 +0000 2018,35.015218,135.677952,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,"I'm at 岚 岚 岚 駅 駅 in Kyoto, Kyoto https://t.co/...",1002383335941865472,None,None,None
326,UTL_tokaido,4880807249,Fri Jun 01 08:27:59 +0000 2018,35.020288,135.678964,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,I'm at Saga Setogawa-cho bus stop in Kyoto cit...,1002466779560665088,None,None,None
407,UTL_tokaido,4880807249,Fri Jun 01 10:05:50 +0000 2018,35.019827,135.676434,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,"When I tried to get on the Kyoto bus, 18 o'clo...",1002491403778314240,None,None,None
942,akahoshi_na,113428108,Sun Jun 03 03:30:36 +0000 2018,35.015218,135.677952,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,"I'm at 岚 岚 岚 駅 駅 in Kyoto, Kyoto https://t.co/...",1003116716933763072,None,None,None
997,boxerimp,128532477,Sun Jun 03 04:12:31 +0000 2018,35.015768,135.677154,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,I'm at Arashiyama Sakura Rice Cake in Kyoto Ci...,1003127263112679425,None,None,None
1186,hiroki_takakura,128572980,Sun Jun 03 06:53:54 +0000 2018,35.015508,135.680825,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,"I'm at Emperor Changqing 嵯峨 Dongling in Kyoto,...",1003167877753638912,None,None,None
1781,tsu___yuuui2525,774583865851596802,Mon Jun 04 02:33:06 +0000 2018,35.018375,135.682340,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,rice! ! (@ Hotel Binaroio Saga Arashiyama in K...,1003464630923153410,None,None,None
1839,jennyypink,285462101,Mon Jun 04 04:49:28 +0000 2018,35.018592,135.676220,ja,"135.775469, 35.267836","135.559007, 34.979538",Ukyo-ku,I'm at Kyoto Arashiyama Music Box Museum in Ky...,1003498949515325440,None,None,None
1840,jujudyworld,324288615,Mon Jun 04 04:52:56 +0000 2018,35.018592,135.676220,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,I'm at Kyoto Arashiyama Music Box Museum in Ky...,1003499824682033159,None,None,None
1854,kuron1539,3175916346,Mon Jun 04 05:15:33 +0000 2018,35.015218,135.677952,ja,"135.775469, 35.267836","135.559007, 34.979538",Kami-ku,"I'm at 岚 岚 岚 駅 駅 in Kyoto, Kyoto https://t.co/...",1003505515085881344,None,None,None


In [27]:
kyt_str = load_obj(path_play + r"\kyt_str")

In [28]:
kyt_idx["0"]

[(135.6753, 34.955205),
 (135.6833, 34.955205),
 (135.6753, 34.96187166666667),
 (135.6833, 34.96187166666667),
 '-1']

### Bangkok

In [29]:
bkk_all = load_obj(path_play+r"\bkk_data2")

In [30]:
bkk_idx, bkk, b_rank, b_sims = bkk_all

In [31]:
bkk[9].head()

,index,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
30802,36524,180291971,RORS_mole,1003523467831103489,"I'm at Gloria Jean's Coffees in Phra Nakhon, B...",2018-06-04 06:26:53,13.746386,100.490552,Grand Palace,"100.487616, 13.740863","100.496715, 13.762037",-1,-1,-1,en
44564,52427,15583257,cowperthwait,1004219564262084608,There is a popular trim of Isuzu in Thailand c...,2018-06-06 04:32:56,13.750582,100.490382,Phra Borom Maha Ratchawang,"100.487616, 13.740863","100.496715, 13.762037",-1,-1,-1,en
56587,66229,160184316,OoMustardCatoO,1004940625182109697,Jamming sesh! I love my friends!!! #bangkok… h...,2018-06-08 04:18:10,13.746461,100.490736,Phra Borom Maha Ratchawang,"100.487616, 13.740863","100.496715, 13.762037",-1,-1,-1,en
58021,67827,240768003,e_shanks,1004990386614493185,3 hippies touring the grand palace @ The Grand...,2018-06-08 07:35:54,13.752410,100.491050,Phra Borom Maha Ratchawang,"100.487616, 13.740863","100.496715, 13.762037",-1,-1,-1,en
62131,72431,383210279,felinenote,1005111152953712640,My Beloved Grandma #RestinPeace… https://t.co...,2018-06-08 15:35:47,13.746366,100.485515,What Arun,"100.480382, 13.736189","100.491927, 13.748122",-1,-1,-1,en


In [32]:
bkk_str = load_obj(path_play+r"\bkk_str")

In [33]:
bkk_str[2]

'<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. What do you eat? <URL>.  @ Riverside… <URL>. Rebirth leaw jaaa (@ Starbucks in Thon Buri, Bangkok) <URL>. After a 15 minute break ... <URL>. <CHECK_IN>. Start! (@ Starbucks in Thon Buri, Bangkok) <URL>. This is my first pos <MENTION> his is my first post.. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>.  \n'

## Merge doc in area

In [34]:
def merge_doc(city_idx, city_str):
    city_doc = {}
    for i in range(len(city_idx)):
        idx = city_idx[str(i)][4]
        text = city_str[i].strip()
        if str(idx) in city_doc:
            city_doc[str(idx)] += text
        else:
            city_doc[str(idx)] = text
    return city_doc

In [35]:
kyt_doc = merge_doc(kyt_idx, kyt_str)

In [36]:
k_bound = len(kyt_doc)
print(k_bound)

32


In [37]:
bkk_doc = merge_doc(bkk_idx, bkk_str)

In [38]:
b_bound = len(bkk_doc)
print(b_bound)

32


## Doc2vec

In [39]:
bkk_doc["-1"]

"<CHECK_IN>.The first dental implant (@ dentist dentist) <URL>. Work with a gang with an apprentice. Same model (@ Bester Studio in Bangkok) <URL>. <CHECK_IN>. Today Friday ... <URL>. <CHECK_IN>. Do you like this? (@ Boom Shop in Bukkhalo, Bangkok) <URL>. <CHECK_IN>.<CHECK_IN>. Come to the students. I do not want to do anything ... <URL>. ... <URL>.Do not panic (@ Ban Somdej Chao Phraya Rajabhat University in Thon Buri, Bangkok) <URL>. More often than myself. (@Bang Somdet Chaopraya University in Thon Buri, Bangkok) <URL>. When will my teeth finish? The doctor scraped the limestone again ... <URL>. Thanks for the inf <MENTION> hanks for the info.. This is a great day for yo <MENTION> his is a great day for you!. I will be traveling # 14062018 @ Temple of Dawn ... <URL>. I do not know what to d <MENTION>  do not know what to do..<CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>. <CHECK_IN>.At eight o'clock (@Bangramat Temple School in Bang Ph

In [40]:
len(bkk_doc)

32

In [41]:
bkk_idx

{'0': [(100.484605, 13.68578),
  (100.49127166666666, 13.68578),
  (100.484605, 13.692446666666665),
  (100.49127166666666, 13.692446666666665),
  '-1'],
 '1': [(100.484605, 13.692446666666665),
  (100.49127166666666, 13.692446666666665),
  (100.484605, 13.699113333333331),
  (100.49127166666666, 13.699113333333331),
  '-1'],
 '10': [(100.484605, 13.752446666666659),
  (100.49127166666666, 13.752446666666659),
  (100.484605, 13.759113333333325),
  (100.49127166666666, 13.759113333333325),
  '5'],
 '100': [(100.52460499999997, 13.752446666666659),
  (100.53127166666663, 13.752446666666659),
  (100.52460499999997, 13.759113333333325),
  (100.53127166666663, 13.759113333333325),
  '100'],
 '101': [(100.52460499999997, 13.759113333333325),
  (100.53127166666663, 13.759113333333325),
  (100.52460499999997, 13.76577999999999),
  (100.53127166666663, 13.76577999999999),
  '100'],
 '102': [(100.52460499999997, 13.76577999999999),
  (100.53127166666663, 13.76577999999999),
  (100.52460499999997

In [42]:
#bkk+kyt
all_doc = []
map_doc = {}
count = 0
for key in bkk_doc:
    text = bkk_doc[key] + "\n"
    all_doc.append(text)
    map_doc[str(count)] = key
    count += 1
for key in kyt_doc:
    text = kyt_doc[key] + "\n"
    all_doc.append(text)
    map_doc[str(count)] = key
    count += 1

In [43]:
len(all_doc)

64

In [44]:
f = open(path_play + "/all_demo_train","w",encoding="utf-8")
for line in all_doc:
    f.write(line)
    #print(line)
f.close()

### Train

In [45]:
train_corpus = list(read_corpus(path_play + "/all_demo_train"))
model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=100)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 19.8 s


In [46]:
#no cope with No data
ranks = []
second_ranks = []
all_sims = []
used = set()
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    all_sims.append(sims)
    
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    #second_ranks.append(sims[1])
    #"""
    tmp = len(sims)-1
    if doc_id < b_bound:
        for i in range(len(sims)):
            if (sims[i][0] >= b_bound):
                if i < tmp:
                    tmp = i
                if (sims[i][0] not in used):
                    second_ranks.append(sims[i])
                    used.add(sims[i][0])
                    break
            if i == (len(sims)-1):
                second_ranks.append(sims[tmp])
    else:
        for j in range(len(sims)):
            if (sims[j][0] < b_bound):
                if j < tmp:
                    tmp = j 
                if (sims[j][0] not in used):
                    second_ranks.append(sims[j])
                    used.add(sims[j][0])
                    break
            if j == (len(sims)-1):
                second_ranks.append(sims[tmp])
    #"""

In [47]:
ranks = []
second_ranks = []
all_sims = []
used = set()
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    all_sims.append(sims)
    
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    #second_ranks.append(sims[1])
    #"""
    tmp = len(sims)-1
    if doc_id < b_bound:
        for i in range(len(sims)):
            if doc_id == 0:
                second_ranks.append((b_bound, 0.999))
                used.add(b_bound)
                break
            elif (sims[i][0] >= b_bound):
                if i < tmp:
                    tmp = i
                if (sims[i][0] not in used):
                    second_ranks.append(sims[i])
                    used.add(sims[i][0])
                    break
            if i == (len(sims)-1):
                second_ranks.append(sims[tmp])
    else:
        for j in range(len(sims)):
            if doc_id == b_bound:
                second_ranks.append((0, 0.999))
                used.add(0)
                break
            elif (sims[j][0] < b_bound):
                if j < tmp:
                    tmp = j 
                if (sims[j][0] not in used):
                    second_ranks.append(sims[j])
                    used.add(sims[j][0])
                    break
            if j == (len(sims)-1):
                second_ranks.append(sims[tmp])
    #"""

In [48]:
print(ranks.count(0))

64


In [49]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (63): «check_in long time no see glove repair degner in yamashina ku kyoto fu renayyxx url come out good night degner in kyoto shi yamashina ku kyoto fu url check_in honma tsuna degner in kyoto city yamashina ku kyoto url north district degner in sanko district kyoto prefecture url check_in check_in tsunagi came here degner in kyoto city yamashina ku kyoto url full ordered tsunagi trying on degner in kyoto city yamashina ku kyoto url check_in check_in returned home degner in kyoto city yamashina ku kyoto url computers degner in kyoto city yamashina ku kyoto url check_in»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d80,n5,w5,mc2,s0.001,t3):

MOST (63, 0.7260802984237671): «check_in long time no see glove repair degner in yamashina ku kyoto fu renayyxx url come out good night degner in kyoto shi yamashina ku kyoto fu url check_in honma tsuna degner in kyoto city yamashina ku kyoto url north district degner in sanko district kyoto prefecture url check_in check_in tsunagi came

In [50]:
len(second_ranks)

64

In [51]:
second_ranks

[(32, 0.999),
 (63, 0.6605046391487122),
 (42, 0.38771945238113403),
 (35, 0.4960516691207886),
 (38, 0.3687206506729126),
 (53, 0.35134464502334595),
 (33, 0.4723461866378784),
 (41, 0.4278568625450134),
 (59, 0.3712407648563385),
 (37, 0.4272187352180481),
 (34, 0.4595564603805542),
 (60, 0.5211611986160278),
 (62, 0.3981069326400757),
 (48, 0.38387298583984375),
 (57, 0.3527936339378357),
 (46, 0.45074737071990967),
 (43, 0.3965013027191162),
 (50, 0.3424202501773834),
 (55, 0.4055660367012024),
 (61, 0.39563101530075073),
 (51, 0.39725953340530396),
 (58, 0.41322439908981323),
 (56, 0.4162883758544922),
 (44, 0.30524492263793945),
 (39, 0.320036381483078),
 (45, 0.33737829327583313),
 (40, 0.36408179998397827),
 (52, 0.326715350151062),
 (49, 0.3216051757335663),
 (36, 0.2930836081504822),
 (47, 0.4028298854827881),
 (54, 0.4435305595397949),
 (0, 0.999),
 (31, 0.6550312042236328),
 (22, 0.4783259630203247),
 (20, 0.751406192779541),
 (16, 0.3521392047405243),
 (30, 0.5223417282104

In [52]:
kyt_idx[str(map_doc["39"])]

[(135.71530000000004, 34.955205),
 (135.72330000000005, 34.955205),
 (135.71530000000004, 34.96187166666667),
 (135.72330000000005, 34.96187166666667),
 '75']

In [53]:
bkk_idx[str(map_doc["31"])]

[(100.57793833333325, 13.76577999999999),
 (100.58460499999991, 13.76577999999999),
 (100.57793833333325, 13.772446666666657),
 (100.58460499999991, 13.772446666666657),
 '222']

## Change color

In [79]:
map_color = {}
bound = max(b_bound, k_bound)
(main_idx, minor_idx) = (bkk_idx, kyt_idx) if b_bound>k_bound else (kyt_idx, bkk_idx)
key_doc = kyt_doc if b_bound>k_bound else bkk_doc
for i in range(bound):
    map_color[str(i)]=color[i%len(color)]

In [103]:
f_loc = open("latlong_all_225_2.txt","w", encoding="utf-8")
color_area = list(print_areas(main_idx))
color_area.sort()
f_loc.write("""{
  "type": "FeatureCollection",
  "features": [
  """)
#pin
pin = []
used_color = []
#write grid
loop = grid_size*grid_size
for i in range(loop):
    idx = color_area.index(main_idx[str(i)][4])
    colorful = map_color[str(idx)]
    #print(colorful)
    if colorful not in used_color:
        used_color.append(colorful)
        lat = (main_idx[str(i)][0][1] + main_idx[str(i)][2][1])/2
        long = (main_idx[str(i)][0][0] + main_idx[str(i)][1][0])/2
        loc = "{lat:" + str(lat) + ",lng:" + str(long) + "}"
        pin.append(loc)
    if colorful != "#56000a":
        latlong = ""
        latlong += str(list(main_idx[str(i)][0])) +", "
        latlong += str(list(main_idx[str(i)][1])) +", "
        latlong += str(list(main_idx[str(i)][3])) +", "
        latlong += str(list(main_idx[str(i)][2])) +", "
        latlong += str(list(main_idx[str(i)][0]))
        f_loc.write("""{
          "type": "Feature",
          "properties": {
            "letter": "o",
            "color": "%s",
            "rank": "15",
            "ascii": "111"
          },
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                %s 
              ]
            ]
          }
        },
        """ % (colorful, latlong))
f_loc.close()

-1  1  1  1  1 -1 -1 -1  8  8  8 -1 -1 -1  14 
-1 -1 45  1 -1 45 45 -1 37 37 -1 -1 -1 -1 -1 
-1 45 45 -1 45 45 -1 37 37 37 -1 -1 42 -1 -1 
45 45 -1 45 45 45 -1 45 45 45 -1 -1 -1 -1 -1 
-1 75 75 75 75 75 75 75 75 75 75 75 -1 -1 -1 
75 75 75 75 75 75 75 75 75 75 75 75 -1 -1 -1 
-1 105 105 -1 -1 105 75 75 75 75 75 75 75 -1 -1 
105 105 105 105 105 105 111 105 113 75 75 75 75 75 75 
105 105 105 105 105 105 105 105 128 129 143 143 143 143 143 
105 105 105 105 139 105 141 142 143 143 143 143 143 143 143 
105 105 105 105 154 105 156 157 158 159 174 143 174 174 174 
105 105 105 105 105 105 105 172 173 174 174 174 174 174 174 
105 181 -1 181 105 105 105 187 188 174 174 191 105 174 174 
-1 181 181 181 -1 105 105 105 203 105 205 105 105 105 105 
-1 -1 -1 -1 214 -1 105 105 105 105 105 105 105 105 105 

 #areas : 32


In [104]:
len(used_color)

32

In [105]:
pin2 = pin.copy()
for j in pin:
    pin2.append(j)

In [106]:
f_loc = open("latlong_all_225_2.txt","a", encoding="utf-8")
for i in range(loop):
    find_key = list(key_doc.keys()).index(str(minor_idx[str(i)][4]))
    if main_idx == bkk_idx:
        find_key += b_bound
    idx = color_area.index(map_doc[str(second_ranks[find_key][0])])
    colorful = map_color[str(idx)]
    #pin
    used_color2 = []
    if colorful not in used_color2:
        used_color2.append(colorful)
        lat = (minor_idx[str(i)][0][1] + minor_idx[str(i)][2][1])/2
        long = (minor_idx[str(i)][0][0] + minor_idx[str(i)][1][0])/2
        loc = "{lat:" + str(lat) + ",lng:" + str(long) + "}"
        pin_idx = used_color.index(colorful)+k_bound
        pin2[pin_idx] = loc
    #print(colorful)
    if colorful != "#56000a":
        latlong = ""
        latlong += str(list(minor_idx[str(i)][0])) +", "
        latlong += str(list(minor_idx[str(i)][1])) +", "
        latlong += str(list(minor_idx[str(i)][3])) +", "
        latlong += str(list(minor_idx[str(i)][2])) +", "
        latlong += str(list(minor_idx[str(i)][0]))
        f_loc.write("""{
          "type": "Feature",
          "properties": {
            "letter": "o",
            "color": "%s",
            "rank": "15",
            "ascii": "111"
          },
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                %s 
              ]
            ]
          }
        },
        """ % (colorful, latlong))
    # remember to delete last ","
f_loc.write("""]
}""")
f_loc.close()

In [107]:
len(pin2)

64

In [108]:
print([", ".join(pin2)])

['{lat:34.95853833333334,lng:135.6793}, {lat:34.965205,lng:135.6793}, {lat:35.01187166666668,lng:135.6793}, {lat:35.051871666666685,lng:135.6793}, {lat:34.97187166666667,lng:135.6873}, {lat:35.01187166666668,lng:135.6873}, {lat:35.03853833333335,lng:135.69530000000003}, {lat:34.965205,lng:135.71130000000005}, {lat:34.965205,lng:135.72730000000007}, {lat:34.99853833333334,lng:135.73530000000005}, {lat:35.01187166666668,lng:135.73530000000005}, {lat:35.01187166666668,lng:135.7433000000001}, {lat:35.01853833333334,lng:135.7433000000001}, {lat:35.025205000000014,lng:135.7433000000001}, {lat:34.98520500000001,lng:135.75130000000007}, {lat:34.99853833333334,lng:135.75130000000007}, {lat:35.005205000000004,lng:135.75130000000007}, {lat:34.98520500000001,lng:135.7593000000001}, {lat:34.99853833333334,lng:135.7593000000001}, {lat:35.005205000000004,lng:135.7593000000001}, {lat:35.01187166666668,lng:135.7593000000001}, {lat:35.01853833333334,lng:135.7593000000001}, {lat:35.025205000000014,lng:13